## Requirements:  
`Create a geographic data visualization that shows the **average** wheat production by country for the years 2020 to 2022 `  
Downloaded world geo.json and using wheat production by year data.  
Create mean average of each years data as new field.  
Fill the country polygon with a color depending on mean value, giving a Choropleth visualisation of country by population    
[Choropleth Map](https://geopandas.org/en/stable/docs/user_guide/mapping.html)  

#Data extraction. Set up the pandas Dataframe object from the given URL and a geoPandasDataFrame  

  

In [ ]:
%matplotlib ipympl
import pandas
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
# create a dataframe containing wheat production data
wheat_production_url = r'https://en.wikipedia.org/wiki/List_of_countries_by_wheat_production'
wheat_query = pandas.read_html(wheat_production_url)
wheat_df = wheat_query[0]

# create a geoDataFrame containing the geometry of the world from previously downloaded geoJson
geoDataFrame = gpd.read_file("C:\\Users\\Trevor\\Documents\\GitHub\\PaletteCohort7\\Trevor\\data\\world_map.geo.json")

Data prep. Wheat DataFrame. Clip excess data before 2020 and remove totals row as it throws everything off.  
Rename the Location to match the GeoDatFrame sovereignt column so we can merge them. Rename USA cell otherwise  
it wont match. **REMEMBER** to capture the output or use `inplace=True` otherwise changes arent recorded ;)

In [ ]:
# remove totals row as this throws everything off and remove every year column before 2020
clipped = wheat_df.iloc[1:, 0:4]
clipped.info()

# get the mean value of rows [2022[1], 2021[1], 2020[1]] and add them to the dataframe
clipped['mean'] = clipped.iloc[:, 1:4].mean(axis=1)

# Location = United States socereignt = 'United States of America'
clipped['Country'] = clipped['Country'].str.replace("United States", "United States of America")
# make the column names match the geoLocationData so merge works

clipped['sovereignt'] = clipped['Country']
# merge favoring the left value
merged_data_frame = pandas.merge(geoDataFrame, clipped, on='sovereignt', how='left')
merged_data_frame.fillna(1, inplace=True)


wheat values == 41 entries(countries)  
geoDataFrame == 242 entries (countries) containing duplicates  
this leaves 201 countries with no mean wheat production value. If we send this to the   
plot() function, it will not render the country, so we have to fill the null values with 1  
**REMEMBER** `inplace=True` to capture the output  
  

In [ ]:
# column= maps color by the column, giving different fill colors to the geometry per value
# legend=True shows the legend of the color values
title_text = "Country By Average Wheat Production 2020-22 (Million Tonnes)"
merged_data_frame.sort_values("mean", ascending=False, inplace=True)



In [ ]:
#column='mean', 
axis = merged_data_frame.plot(legend=True, 
                         column='mean',
                              cmap='OrRd',
                              aspect=1, 
                              edgecolor='black', 
                              figsize=(10, 6)
                              ).set_title(title_text)

plt.show()